## Лабораторная работа 1. Морфологический парсер mystem

**Задание 1.** Изучите документацию и лицензию (!) морфологического парсера mystem от Yandex: https://yandex.ru/dev/mystem.

**Задание 2.** Установите `pymystem3` – интерфейс к mystem на Python: https://pypi.org/project/pymystem3.

(!) Обратите внимание, что у конструктора объекта Mystem() есть параметры.

**Задание 3.** Выпишите с какими параметрами запускается морфологический анализатор.

*Ваш ответ:*

**а)** Придумайте и запишите примеры предложений со словами не из словаря. Приведите их полученные морфологические разборы.

*Ваш ответ:*

**б)** Применяется ли контекстное снятие омонимии при морфологическом разборе?

*Ваш ответ:*

In [73]:
from pymystem3 import Mystem
text = "Я решил запушить лонг, но меня закидали флешками и я был в пермоблайнде"
m = Mystem(disambiguation=True)
lemmas = m.lemmatize(text)
print(''.join(lemmas))

я решать запушить лонг, но я закидывать флешка и я быть в пермоблайнде



**Задание 4.** Напишите функцию `parse_text()`, на вход которой поступает текст (в виде строки), а на выходе формируется структура данных, содержащая для каждого слова входного текста следующую информацию:
- исходную словоформу (wordform);
- нормальную форму слова (лемму) (norm, lemma);
- часть речи (part of speech, POS);
- другую грамматическую информацию, выдаваемую mystem;
- признак, присутствует ли слово в словаре mystem.

Функция должна выбирать наиболее вероятный вариант морфологического разбора слова.

In [74]:
from pymystem3 import Mystem

def parse_text(text):
    m = Mystem(disambiguation=True)
    analysis = m.analyze(text)
    result = []

    for item in analysis:
        if 'analysis' in item and item['analysis']:
            best_parse = item['analysis'][0]
            result.append({
                'wordform': item['text'],
                'lemma': best_parse.get('lex', item['text']),
                'pos': best_parse['gr'].split(',')[0] if 'gr' in best_parse else None,
                'gram_info': best_parse.get('gr', ''),
                'in_dict': True
            })
        else:
            result.append({
                'wordform': item['text'],
                'lemma': item['text'],
                'pos': None,
                'gram_info': '',
                'in_dict': False
            })
    return result

**Задание 5.** Напишите функцию `save_morph_results()`, сохраняющую структуру данных, получаемую функцией `parse_text()`, в текстовый файл формата JSON.

In [75]:
import json

def save_morph_results(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

**Задание 6.** Напишите функцию `get_dictionary()`, на вход которой поступает текст (в виде строки), а на выходе формируется словарь,
включающий все уникальные слова текста и содержащий для каждого слова следующую информацию:
- нормальную форму слова;
- часть речи;
- частоту слова в тексте;
- все варианты словоформ в тексте с данной нормальной формой.

In [76]:
from collections import defaultdict

def get_dictionary(text):
    parsed = parse_text(text)
    word_freq = Counter()
    lemmas_info = {}

    for item in parsed:
        word = item['wordform'].lower()
        lemma = item['lemma']
        pos = item['pos']

        if item['in_dict']:
            word_freq[lemma] += 1
            if lemma not in lemmas_info:
                lemmas_info[lemma] = {'pos': pos, 'forms': set()}
            lemmas_info[lemma]['forms'].add(word)

    dictionary = {}
    for lemma, info in lemmas_info.items():
        dictionary[lemma] = {
            'pos': info['pos'],
            'frequency': word_freq[lemma],
            'wordforms': list(info['forms'])
        }

    return dictionary

**Задание 7.** Напишите функцию `save_dictionary()`, сохраняющую предыдущую структуру данных в текстовый файл формата JSON. Слова в файле должны быть упорядочены по убыванию частоты.

In [77]:
def save_dictionary(dictionary, filename):
    sorted_dict = dict(sorted(dictionary.items(), key=lambda x: x[1]['frequency'], reverse=True))
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(sorted_dict, f, ensure_ascii=False, indent=2)

**Задание 8.** Напишите функцию `get_non_mystem_dict()`, на вход которой поступает структура данных, получаемая функцией `parse_text()`, а на выходе формируется словарь, содержащий уникальные слова текста, отсутствующие в словаре mystem, вместе с частотой слова в тексте.

In [78]:
def get_non_mystem_dict(parsed_data):
    non_dict_words = Counter()
    for item in parsed_data:
        if not item['in_dict'] and item['wordform'].strip():
            non_dict_words[item['wordform']] += 1
    return non_dict_words

**Задание 9.** Напишите функцию `save_non_mystem_dict()`, сохраняющую структуру данных, получаемую функцией `get_non_mystem_dict()`, в текстовый файл формата TSV (tab-separated values). Слова в файле должны быть упорядочены по убыванию частоты.

In [79]:
def save_non_mystem_dict(non_dict_words, filename):
    sorted_words = non_dict_words.most_common()
    with open(filename, 'w', encoding='utf-8') as f:
        for word, freq in sorted_words:
            f.write(f"{word}\t{freq}\n")

**Задание 10.** Напишите функцию `get_pos_distribution()`, на вход которой поступает словарь, формируемый функцией `get_dictionary()`, а на выходе выдается структура данных, содержащая частотное распределение частей речи в словаре со следующими значениями


|часть речи|количество уникальных слов|общее количество слов|
| -------- | ------------------------ | ------------------- |

In [80]:
def get_pos_distribution(dictionary):
    pos_count = defaultdict(lambda: {'unique': 0, 'total': 0})
    for lemma, info in dictionary.items():
        pos = info['pos']
        pos_count[pos]['unique'] += 1
        pos_count[pos]['total'] += info['frequency']
    return pos_count

**Задание 11.** Проведите эксперименты с разработанными функциями:
- скачайте 10 файлов с текстами разных жанров и разного размера (например, произведения классиков, современных писателей, новостные статьи, научные статьи и т.п.). *Учитывайте кодировку* – все файлы должны быть в UTF-8;
- обработайте файлы при помощи функций `parse_text()`, `get_dictionary()` и `get_non_mystem_dict()`, и сохраните результаты в текстовых файлах при помощи функций `save_morph_results()`, `save_dictionary()` и `save_non_mystem_dict()`. Измеряйте время запуска функций! (см. следующий пункт);
- заполните следующую таблицу:

|Файл|Размер, байт|Размер текста (кол-во слов)|Размер словаря (кол-во уникальных слов)|Время работы get_dictionary(), сек.|
|----|------------|---------------------------|---------------------------------------|-----------------------------------|
- для самого большого словаря постройте частотное распределение слов:
  - по оси ординат – частота,
  - по оси абсцисс – слова, упорядоченные по убыванию частоты (по-другому, ранги слов);
- постройте график зависимость времени морфологического анализа от размера текстового файла;
- распределение частей речи, полученное функцией `get_pos_distribution()`, выведите на экран в виде таблицы и графика.


In [81]:
import os
import time
import json
import matplotlib.pyplot as plt
from collections import Counter
from pymystem3 import Mystem

files = [
    'text1.txt', 'text2.txt', 'text3.txt',
    'text4.txt', 'text5.txt'
]

results = []
all_dictionaries = []

for file in files:
    if not os.path.exists(file):
        print(f"Файл {file} не найден, пропускаем")
        continue

    with open(file, 'r', encoding='utf-8') as f:
        text = f.read()

    start_time = time.time()
    dictionary = get_dictionary(text)
    elapsed_time = time.time() - start_time

    results.append({
        'file': file,
        'size_bytes': os.path.getsize(file),
        'word_count': len(text.split()),
        'unique_words': len(dictionary),
        'time': elapsed_time
    })

    all_dictionaries.append(dictionary)

    save_dictionary(dictionary, f'dict_{file}.json')

    parsed_data = parse_text(text)
    save_morph_results(parsed_data, f'parsed_{file}.json')

    non_dict_words = get_non_mystem_dict(parsed_data)
    save_non_mystem_dict(non_dict_words, f'non_dict_{file}.tsv')

print("Результаты анализа файлов:")
for res in results:
    print(f"{res['file']}: {res['size_bytes']} байт, {res['word_count']} слов, "
          f"{res['unique_words']} уникальных слов, время: {res['time']:.2f} сек.")

largest_dict = max(all_dictionaries, key=len)

frequencies = sorted([item['frequency'] for item in largest_dict.values()], reverse=True)
plt.figure(figsize=(12, 6))
plt.plot(frequencies)
plt.title('Частотное распределение слов')
plt.xlabel('Ранг слова')
plt.ylabel('Частота')
plt.savefig('word_frequency.png')
plt.show()

sizes = [res['word_count'] for res in results]
times = [res['time'] for res in results]
plt.figure(figsize=(10, 6))
plt.scatter(sizes, times)
plt.title('Зависимость времени анализа от размера текста')
plt.xlabel('Количество слов в тексте')
plt.ylabel('Время анализа (сек)')
plt.savefig('time_vs_size.png')
plt.show()

pos_distribution = get_pos_distribution(largest_dict)
print("Распределение частей речи:")
for pos, counts in pos_distribution.items():
    print(f"{pos}: {counts['unique']} уникальных слов, {counts['total']} всего слов")

pos_names = list(pos_distribution.keys())
unique_counts = [pos_distribution[pos]['unique'] for pos in pos_names]

plt.figure(figsize=(12, 6))
plt.bar(pos_names, unique_counts)
plt.title('Распределение частей речи')
plt.xlabel('Часть речи')
plt.ylabel('Количество уникальных слов')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('pos_distribution.png')
plt.show()

KeyboardInterrupt: 